In [10]:
from nltk import word_tokenize,pos_tag

In [11]:
from nltk.tokenize import sent_tokenize

In [12]:
para = """
This cute little set is not only sturdy and realistic, it was also a wonderful introduction to
preparing food for our 3 year old daughter. Ever since her Grandpa bought this for her, she's
made everything from a cheese sandwhich to a triple decker salami club! She had so much fun
playing with this toy, that she started to become interested in how I prepared meals. She now is
very eager to spread peanut butter and jam, layer turkey and cheese and help mix cake batter.
A very cute gift to give a girl or boy.
       """

In [13]:
sents = sent_tokenize(para)

In [14]:
sents[0]

'\nThis cute little set is not only sturdy and realistic, it was also a wonderful introduction to\npreparing food for our 3 year old daughter.'

In [15]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

In [16]:
text ="The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head."

text_token = word_tokenize(text)
text_pos = pos_tag(text_token)
text_pos[0]
text_pos_norm =[]
new_text =[]
for t in text_pos:
    lem = lemma.lemmatize(t[0])
    text_pos_norm.append((lem,t[1]))
    new_text.append(lem)
    
' '.join(new_text)

u'The best scene in the movie wa when Gerardo is trying to find a song that keep running through his head .'

In [23]:
from nltk.wsd import lesk

def create_syn_set_annotation(text_pos_normalized,text):
    text_syn_set_list =[]
    for t in text_pos_normalized:
        if t[1][:2] =='NN':
            if lesk(text,t[0],'n'):
                text_syn_set_list.append(lesk(text,t[0],'n'))
        elif t[1][:2] =='VB':
            if lesk(text,t[0],'v'):
                text_syn_set_list.append(lesk(text,t[0],'v'))
        elif t[1][:2] =='RB':
            if lesk(text,t[0],'r'):
                text_syn_set_list.append(lesk(text,t[0],'r'))
        elif t[1][:2] =='JJ':
            if lesk(text,t[0],'a'):
                text_syn_set_list.append(lesk(text,t[0],'a'))
        else:
            continue
    return text_syn_set_list
syn_list = create_syn_set_annotation(text_pos_norm,new_text)

In [24]:
syn_list

[Synset('good.a.01'),
 Synset('scenery.n.01'),
 Synset('movie.n.01'),
 Synset('be.v.12'),
 Synset('try_on.v.01'),
 Synset('find.v.13'),
 Synset('sung.n.01'),
 Synset('retain.v.02'),
 Synset('tend.v.01'),
 Synset('head.n.25')]

In [25]:
import numpy as np
from nltk.corpus import sentiwordnet as swn

def polarity_score(text_syn_set_list):
    limp =[]
    for s in text_syn_set_list:
        pos_score = swn.senti_synset(s.name()).pos_score()
        neg_score = swn.senti_synset(s.name()).neg_score()
        neut_score = swn.senti_synset(s.name()).obj_score()
        polarity_score = 0
        if pos_score > neg_score and pos_score > neut_score:
            polarity_score = 1
        elif neg_score > pos_score and neg_score > neut_score:
            polarity_score = -1
        else:
            polarity_score = 0
        limp.append(polarity_score)
    arr = np.array(limp)
    return round(arr.mean(),1)

In [26]:
polarity_score(syn_list)

0.2

In [27]:
def sentiment_analysis(text):
    text_token = word_tokenize(text)
    text_pos = pos_tag(text_token)
    text_pos_norm =[]
    new_text_list =[]
    for t in text_pos:
        lem = lemma.lemmatize(t[0])
        text_pos_norm.append((lem,t[1]))
        new_text_list.append(lem)
    new_text = ' '.join(new_text_list)
    syn_set_list = create_syn_set_annotation(text_pos_norm,new_text)
    polarity = polarity_score(syn_set_list)
    return polarity


In [28]:
sentiment_analysis(text)

0.2

In [29]:
sentiment_analysis('A very, very, very slow-moving, aimless movie about a distressed, drifting young man.')

-0.1

In [30]:
sentiment_analysis('Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.')

-0.2

In [31]:
def subjectivity_score(text_syn_set_list):
    limp = []
    for s in text_syn_set_list:
        limp.append(swn.senti_synset(s.name()).obj_score())
    arr = np.array(limp)
    return round(np.mean(limp),1)
objectivity_score(syn_list)

0.8